In [96]:
from bs4 import BeautifulSoup
import requests
import re
from collections import Counter
import json
import time

In [3]:
#Download site
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'}

try:
    dict_response = requests.get("https://www.4icu.org/reviews/index0001.htm", headers=headers)
    dict_txt = dict_response.content
    if (dict_response.status_code == requests.codes.ok):
        pass
except Exception as e:
    print(e)

In [6]:
#Parse HTML
soup = BeautifulSoup(dict_txt,"html.parser")

#Get all the table that contains links
list_of_tables = soup.findAll('ul',{'class' : 'pagination pagination-sm'})

#Extract Links
link_list = []
for tag in list_of_tables[0]:
    if tag != None:
        link_list.append(tag.find('a')['href'])
        

In [51]:
university_pages = []
for page in link_list:
    try:
        dict_response = requests.get("https://www.4icu.org/reviews/{}".format(page), headers=headers)
        dict_txt = dict_response.content
        if (dict_response.status_code == requests.codes.ok):
            soup = BeautifulSoup(dict_txt,"html.parser")
            tables = soup.findAll('table', {'class' : 'table table-hover text-left'})
            for td_ in tables[0].findAll('td'):
                link = td_.find('a')
                if link != None:
                    university_pages.append(link['href'])
        else:
            print(dict_response.status_code)
    except Exception as e:
        print(e)
        

13774

In [100]:
all_uni_objects = []
for uni_page in university_pages:
    exponent = 0
    while(True):
        try: 
            test_response = requests.get("https://www.4icu.org{}".format(uni_page), headers=headers)
            if test_response.status_code == requests.codes.ok:
                soup = BeautifulSoup(test_response.content,"html.parser")
                uni_aliases = []
                foundingYear = None
                for td_ in soup.findAll('table', {'class' : 'table borderless'})[0].findAll('td'):
                    span = td_.find('span')
                    abbr = td_.find('abbr')
                    if span != None:
                        if span['itemprop'] in ['name', 'alternateName']:
                            uni_aliases.append(span.find('strong').get_text())
                        elif span['itemprop'] in ['foundingDate']:
                            foundingYear = span.find('strong').get_text()
                    if abbr != None:
                        uni_aliases.append(abbr.find('strong').get_text())

                uni_object = {"uni_aliases": uni_aliases, "founding_year" : foundingYear}
                all_uni_objects.append(uni_object)
                break
            else:
                #exponential back off
                time.sleep(2^exponent)
                exponent = exponent + 1
        except Exception as e:
            print(e)
    
            
            


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [130]:
with open("data/all_universities_cleaned.json", "w") as outfile:
    json.dump(uni_json,outfile)

In [125]:
uni_counter = Counter()
for uni in uni_json['all_universities']:
    try:
        uni['uni_aliases'].remove(to_remove)
    except:
        pass
    for alias in uni['uni_aliases']:
        uni_counter[alias] += 1


{'all_universities': [{'uni_aliases': ["3iL École d'ingénieurs",
    '3iL School of Engineering'],
   'founding_year': '1987'},
  {'uni_aliases': ['A.T. Still University', 'ATSU'], 'founding_year': '1892'},
  {'uni_aliases': ['Aalborg Universitet', 'Aalborg University', 'AAU'],
   'founding_year': '1974'},
  {'uni_aliases': ['Aalto-yliopisto', 'Aalto University', 'A?'],
   'founding_year': '2010'},
  {'uni_aliases': ['Aarhus Universitet', 'Aarhus University', 'AU'],
   'founding_year': '1928'},
  {'uni_aliases': ['Aba Teachers University', '阿坝师范学院', 'ATU'],
   'founding_year': '1978'},
  {'uni_aliases': ['Abasyn University', 'اباسین یونیورسٹی', 'AU'],
   'founding_year': '2007'},
  {'uni_aliases': ['Abbottabad University of Science and Technology',
    'ایبٹ آباد یونیورسٹی آف سائنس اینڈ ٹیکنالوجی',
    'AUST'],
   'founding_year': '2015'},
  {'uni_aliases': ['Abdul Wali Khan University Mardan',
    'عبدالولی خان یونیورسٹی مردان',
    'AWKUM'],
   'founding_year': '2009'},
  {'uni_alias

In [ ]:
test_response = requests.get("https://www.4icu.org{}".format(university_pages[2000]), headers=headers)

In [ ]:
soup = BeautifulSoup(test_response.content,"html.parser")

In [ ]:
uni_aliases = []
foundingYear = None
for td_ in soup.findAll('table', {'class' : 'table borderless'})[0].findAll('td'):
    span = td_.find('span')
    abbr = td_.find('abbr')
    if span != None:
        if span['itemprop'] in ['name', 'alternateName']:
            uni_aliases.append(span.find('strong').get_text())
        elif span['itemprop'] in ['foundingDate']:
            foundingYear = span.find('strong').get_text()
    if abbr != None:
        uni_aliases.append(abbr.find('strong').get_text())

uni_object = {"uni_aliases": uni_aliases, "founding_year" : foundingYear}
uni_object



{'uni_aliases': ['Dr K.N. Modi University',
  'डॉ। के.एन. मोदी यूनिवर्सिटी',
  'DKNMU'],
 'founding_year': '2010'}